In [1]:
from sentence_transformers import SentenceTransformer

In [2]:
import pandas as pd

df = pd.read_csv('PMCXML6.tsv', delimiter='\t')
df.head()

,fileid,filename,AccessionID,pmid,MID,title,registries,tag,isNCT,haveAlready,isReal,NCTclean,section,altsection,noLic,NLMPT,MTPT,TPscore,Country,year
0,925751,oa_comm_xml.incr.2024-06-28.tar.gz:PMC010xxxxx...,PMC10515754,37745325,PMC10515754,article > back > ack > p > ext-link,NCT00001360,ext-link,1,0,1,NCT00001360,Other,Other,0,preprint,NaN,NaN,USA,2024.0
1,925752,oa_comm_xml.incr.2024-06-28.tar.gz:PMC010xxxxx...,PMC10749708,37622521,PMC10749708,article > body > INTRODUCTION > p,NCT02770716,p,1,0,1,NCT02770716,Introduction,Introduction,0,NaN,retrospective studies|multicenter study,0.999005,USA,2023.0
2,925753,oa_comm_xml.incr.2024-06-28.tar.gz:PMC010xxxxx...,PMC10749708,37622521,PMC10749708,article > body > INTRODUCTION > p,NCT01143246,p,1,0,1,NCT01143246,Introduction,Introduction,0,NaN,retrospective studies|multicenter study,0.983015,USA,2023.0
3,925754,oa_comm_xml.incr.2024-06-28.tar.gz:PMC010xxxxx...,PMC10749708,37622521,PMC10749708,article > body > INTRODUCTION > p,NCT02770716,p,1,0,1,NCT02770716,Introduction,Introduction,0,NaN,retrospective studies|multicenter study,0.999005,USA,2023.0
4,925755,oa_comm_xml.incr.2024-06-28.tar.gz:PMC010xxxxx...,PMC10749708,37622521,PMC10749708,article > body > INTRODUCTION > p,NCT01143246,p,1,0,1,NCT01143246,Introduction,Introduction,0,NaN,retrospective studies|multicenter study,0.983015,USA,2023.0


In [3]:
# number_per_class = 17000
# selected_indices = (
#     df.groupby('section')
#     .apply(lambda x: x.sample(number_per_class, random_state=42))
#     .index.get_level_values(1)
# )
# df = df.loc[selected_indices]
# data_size = f'{number_per_class}_balanced'

In [4]:
x = []
sections = []
data_size = 177918
for ind, row in df.iterrows():
    if len(x) >= data_size:
        break
    parts = row['title'].split('>')
    desired_text = parts[2].strip()
    if desired_text != "":
        x.append(desired_text)
        sections.append({row['section'], row['altsection']})

len(x)
    

177918

In [5]:
model_name = "CosineSimilarityLoss"
fine_tuned_model = SentenceTransformer("models/fine_tuned_sentence_bert_model_"+ model_name)

embeddings = fine_tuned_model.encode(x)

In [6]:
import umap
import numpy as np
from sklearn.cluster import DBSCAN

# Assuming embeddings is a (177919, 300) NumPy array
reducer = umap.UMAP(n_components=10, n_neighbors=30, min_dist=0.1, metric='cosine')  
reduced_embeddings = reducer.fit_transform(embeddings)

/jet/home/slin23/.local/lib/python3.12/site-packages/umap/spectral.py:548: UserWarning: Spectral initialisation failed! The eigenvector solver
failed. This is likely due to too small an eigengap. Consider
adding some noise or jitter to your data.

Falling back to random initialisation!
  warn(


In [7]:
from sklearn.cluster import HDBSCAN

min_cluster_size = 5000

clustering = HDBSCAN(min_cluster_size=min_cluster_size).fit(reduced_embeddings)

clustering.labels_
clustering

HDBSCAN(min_cluster_size=5000)

In [8]:
np.unique(clustering.labels_)

array([-1,  0,  1,  2,  3,  4,  5,  6,  7])

In [9]:
name = "clustering/" + model_name.lower() + f"/clustering_labels_{min_cluster_size}_{data_size}"
np.save(name + ".npy", clustering.labels_)


In [10]:
import numpy as np
import pandas as pd
# name = "clustering/" + model_name.lower() + f"/clustering_labels_{min_cluster_size}_{data_size}"
clustering_labels = np.load(name + ".npy")

In [11]:
unique_labels = np.unique(clustering_labels)

data_dict = {}

for label in unique_labels:
    inds = np.where(clustering_labels == label)[0] 
    data_dict[f"Cluster_{label}"] = [x[ind] for ind in inds] 

df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in data_dict.items()]))

# 存入 Excel
df.to_excel(name + ".xlsx", index=False)
pd.read_excel(name + ".xlsx")

,Cluster_-1,Cluster_0,Cluster_1,Cluster_2,Cluster_3,Cluster_4,Cluster_5,Cluster_6,Cluster_7
0,ack,article-meta,INTRODUCTION,Materials and methods,METHODS,Results,ref-list,TTVR,Discussion
1,ack,article-meta,INTRODUCTION,Materials and methods,METHODS,Results,ref-list,TTVR,Discussion
2,ack,article-meta,INTRODUCTION,Materials and methods,METHODS,Results,ref-list,ETHICAL APPROVAL,DISCUSSION
3,Unlocking the potential of MSCs and exosomes i...,article-meta,INTRODUCTION,Materials and methods,METHODS,Results,ref-list,ETHICAL APPROVAL,DISCUSSION
4,Unlocking the potential of MSCs and exosomes i...,article-meta,Introduction,Materials and methods,Methods,Results,ref-list,Financial & competing interests disclosure,Discussion
...,...,...,...,...,...,...,...,...,...
97456,2. Patients and methods,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97457,2. Patients and methods,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97458,2. Patients and methods,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97459,4. Discussion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
